In [2]:
#subrountine to fine the orthogonal projection of v onto u
def proj(v,u):
    a = (v.dot_product(u)/(norm(u)^2)) * u
    return a
u1 = vector([3,-2,4])
u2 = vector([2,1,3])
k = proj(u2,u1)
norm(k).n()

2.97112541083283

In [3]:
def GS(S):
    n = len(S)
    E = [S[0]/norm(S[0]).n()]
    for k in range (1,n):
        q = S[k] - sum([proj(S[k],E[i]) for i in range(k)])
        #range(k) =[0,...,k-1]
        E.append(q/norm(q).n())
    return E

In [4]:
v1 = vector([2,-1,3,1])
v2 = vector([2,2,4,0])
v3 = vector([1,-2,0,-1])
v4 = vector([4,5,5,-1])
L=[v1,v2,v3,v4]
L

[(2, -1, 3, 1), (2, 2, 4, 0), (1, -2, 0, -1), (4, 5, 5, -1)]

In [5]:
#orthogonal basis of L
G1 = GS(L)
G1

[(0.516397779494322, -0.258198889747161, 0.774596669241483, 0.258198889747161),
 (0.0403238919272755, 0.887125622400063, 0.362915027345480, -0.282267243490929),
 (0.362915027345480, -0.282267243490929, -0.0403238919272757, -0.887125622400063),
 (0.774596669241484, 0.258198889747161, -0.516397779494322, 0.258198889747161)]

In [6]:
#Reduction Step
def reductionstep(B,G):
    n = len(B)
    E=[B[0]]
    for i in range(1,n):
        for j in srange(i-1,0,-1, include_endpoint=True):
            cij = round(B[i].dot_product(G[j])/(norm(G[j])^2))
            B[i] = B[i] - cij * B[j]
        E.append(B[i])
    return E    

In [7]:
reductionstep(L,G1)

[(2, -1, 3, 1), (-6, 6, -8, -4), (17, -7, 25, 6), (-47, 14, -76, -19)]

In [54]:
#computes the GS orthogonalization and then test if the matrix B is delta-reduced or not
#b_1,...b_n, b_i\in\ZZ^m
def is_LLL_reduced(B,c = 2):
    m = B.ncols() #Return the number of columns of matrices in this space
    n = B.nrows() #Return the number of rows of matrices in this space
    U = matrix(RR, m, m)
    O = matrix(RR, n, m)
    for i in range(0,m):
        U[i,i] = 1
        O.set_column(i,B.column(i)) #Sets the entries of column i to the entries of B.column(i)
        for j in range(0,i):
            U[j,i] = (B.column(i)*O.column(j))/( O.column(j) * O.column(j)) #\mu_{j,i}
            O.set_column(i,O.column(i) - U[j,i] * O.column(j)) 
    
    for i in range(0,m-1):
        if O.column(i)*O.column(i) > c*O.column(i+1)*O.column(i+1):
            return False
    return True
#U is used to compute c_{i,j} in the GS process
#O is the GS orthogonal basis of B
#the second for ... is used to check the second condition of the LLL basis


In [63]:
#the LLL algorithm
#the second for... in the reduction step
def reducestep(i, B, U):
    j=i-1
    while j>=0:
        B.set_column(i,B.column(i)- round(U[j,i])*B.column(j))
        #update new U[i,j]:
        U.set_column(i,U.column(i)- round(U[j,i])*U.column(j))
        j = j - 1

def LLL(B, c = 2):
    m = B.ncols()
    n = B.nrows()
    U = matrix(RR, m, m)
    O = matrix(RR, n, m)
    #The GS process
    for i in range(0,m):
        U[i,i] = 1
        O.set_column(i,B.column(i)) #Sets the entries of column i to the entries of B.column(i)
        for j in range(0,i):
            U[j,i] = (B.column(i)*O.column(j))/( O.column(j) * O.column(j)) #\mu_{j,i}
            O.set_column(i,O.column(i) - U[j,i] * O.column(j))
            #O is the GS orthogonal basis of B
            reducestep(i, B, U) # the first for... in the reduction step
            
    i = 0
    #swap step
    while i < m - 1:
        if O.column(i)*O.column(i) <= c*O.column(i+1)*O.column(i+1):
            #the sesond condition in the definition of LLL basis - Remark 3
            i = i + 1
        else:
            #swap i <-> i + 1
            O.set_column(i+1,O.column(i+1) + U[i,i+1]*O.column(i))
            #update the GS of column i+1
            #new a_{i+1} = \tilde(b_i) - <b_i,\tilde(a_i)>/<\tilde(a_i),\tilde(a_i)>\tilde(a_i)
            #new a_i = \tilde(b_{i+1}) + ((<b_{i+1},\tilde(b_{i})>/<\tilde(b_{i}),\tilde(b_{i})>))\tilde(b_i)
            #update c_{i,i+1} = <a_i,\tilde(a_{i}+1)>/<\tilde(a_{i}+1),\tilde(a_{i}+1)>
            #how ever at this step we haven't swap yet
            # = <b_i,new b_{i+1}>/<new b_{i+1},new b_{i+1}>
            U[i,i] = (B.column(i)*O.column(i+1))/(O.column(i+1)*O.column(i+1))
            #swap latter
            #old U[i,i]=1
            U[i+1,i] = 1 #this is the new U[i,i]
            U[i,i+1] = 1 #this is the new U[i,i]
            U[i+1,i+1] = 0 #equal to 0 because it gonna be perpendicular, <,>=0
            
            #swapppppp
            O.set_column(i,O.column(i)-U[i,i]*O.column(i+1))
            U.swap_columns(i,i+1)
            O.swap_columns(i,i+1) #swap a_i, a_{i+1} above
            B.swap_columns(i,i+1)
            for k in range(i+2,m):
                U[i,k] = (B.column(k)*O.column(i))/(O.column(i)*O.column(i))
                U[i+1,k] = (B.column(k)*O.column(i+1))/(O.column(i+1)*O.column(i+1))
            
            if abs(U[i,i+1]) > 0.5: 
                reducestep(i+1,B,U)
                i = max(i-1,0)
        return B 

In [64]:
#test 
A = random_matrix(ZZ,10)
res = LLL(A)
res

[  1  14   0   4   0  -1   0   0   0   2]
[  2   2  -1  -1   0  -1   0  -1   1   3]
[  1   1  -3   0   1   0   2  -4  -1   0]
[ -2  -5  -1   7   0   3  -1  -2  -1   2]
[  2  18  -1   1   0  -3   2   3   4   0]
[  0  -1   1  30  -2  -2  -3 -11  -9   6]
[ -3  19   2   0   1  -4  -1   0   0   2]
[  0   4  -1   1  -1   0   1  -1  -1   2]
[ -1   3  -3   1   1  -1   1   3   1   2]
[  0   0 -31   1   0   0   1   0  -2  -1]

In [65]:
is_LLL_reduced(res,c=2)

False

In [ ]:
#it kept output False and it already took me 2 days...
#I haven't figured it out why it didn't work
#I suppose I gonna turn back to it latter...